## Import libraries

In [175]:
import warnings
warnings.filterwarnings("ignore")

In [176]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

## Anomaly-based IDS - Infiltration

### Generate the port-scan datasets for unknown attack detection

In [296]:
df=pd.read_csv('CICIDS2017_sample_km.csv')

In [297]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,2.565000,-0.439152,-0.005817,-0.009418,-0.040156,-0.007139,-0.211382,-0.210703,-0.153699,-0.164849,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
1,2.759600,-0.439341,-0.009819,-0.009418,-0.051263,-0.007139,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
2,2.725909,-0.439340,-0.009819,-0.009418,-0.051863,-0.007139,-0.246240,-0.310140,-0.229468,-0.167112,...,0.002556,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
3,2.967710,-0.439345,-0.009819,-0.010421,-0.051263,-0.007142,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
4,2.683412,-0.430518,-0.008485,-0.008416,-0.045960,-0.007137,-0.180706,-0.210703,-0.151549,-0.097234,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25485,-0.417175,2.529503,0.022195,0.015647,0.612863,-0.007073,1.333548,-0.210703,1.066630,1.056629,...,0.002545,0.847478,0.135514,0.572316,0.898203,1.023440,4.126922,1.900294,0.555568,9
25486,-0.417175,2.871035,0.015525,0.010634,0.403729,-0.007086,0.693545,-0.210703,0.860244,0.661178,...,0.002545,0.773475,-0.099716,0.420533,0.907276,1.967152,-0.109145,1.892086,2.006554,9
25487,-0.417175,1.922710,0.042204,0.028681,1.412575,-0.007038,1.746272,-0.210703,1.609455,1.434458,...,0.002545,4.152361,7.829669,4.707039,0.882819,1.201179,5.539049,1.904398,0.444283,9
25488,-0.417175,1.000343,-0.003150,-0.002400,0.141060,-0.007121,1.053285,-0.210703,1.191111,1.068873,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,9


In [298]:
df.Label.value_counts()

,count
Label,
0,10226
4,4619
10,3178
2,2564
1,1966
12,1507
14,652
3,208
7,155


In [299]:
df1 = df[df['Label'] != 3]
df1['Label'][df1['Label'] > 0] = 1
df1.to_csv('./CICIDS2017_sample_km_without_infiltration.csv',index=0)

In [300]:
df2 = df[df['Label'] == 3]
df2['Label'][df2['Label'] == 3] = 1
df2.to_csv('./CICIDS2017_sample_km_infiltration.csv',index=0)

### Read the generated datasets for unknown attack detection

In [301]:
df1 = pd.read_csv('./CICIDS2017_sample_km_without_infiltration.csv')
df2 = pd.read_csv('./CICIDS2017_sample_km_infiltration.csv')

In [302]:
features = df1.drop(['Label'],axis=1).dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df2[features] = df2[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [303]:
df1.Label.value_counts()

,count
Label,
1,15056
0,10226


In [304]:
df2.Label.value_counts()

,count
Label,
1,208


In [305]:
df2p=df1[df1['Label']==0]
df2pp=df2p.sample(n=None, frac=208/10226, replace=False, weights=None, random_state=None, axis=0)
df2=pd.concat([df2, df2pp])

In [306]:
df2.Label.value_counts()

,count
Label,
1,208
0,208


In [307]:
df = pd.concat([df1, df2])

In [308]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3.151985,-0.504589,-0.009518,-0.011606,-0.033642,-0.009666,-0.148247,-0.084465,-0.081178,-0.147365,...,-0.994008,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
1,3.382380,-0.504767,-0.013408,-0.011606,-0.037097,-0.009666,-0.178735,-0.084465,-0.132058,-0.166535,...,-0.994008,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
2,3.342492,-0.504766,-0.013408,-0.011606,-0.037283,-0.009666,-0.178735,-0.199375,-0.145810,-0.149865,...,0.787413,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
3,3.628770,-0.504771,-0.013408,-0.012608,-0.037097,-0.009668,-0.178735,-0.084465,-0.132058,-0.166535,...,-0.994008,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
4,3.292178,-0.496464,-0.012111,-0.010604,-0.035447,-0.009663,-0.121416,-0.084465,-0.079345,-0.072662,...,-0.994008,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9528,-0.378803,1.200057,0.003446,0.000420,-0.007371,-0.008096,0.510314,-0.199375,0.086283,0.383034,...,-0.994008,0.081629,0.244216,0.274935,0.038307,-0.106637,-0.128271,-0.122260,-0.089704,0
10141,-0.404058,-0.503706,-0.013408,-0.010604,-0.036256,-0.009610,-0.177516,0.413480,-0.070177,-0.235992,...,0.787413,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0
3923,-0.402309,1.196328,-0.004332,-0.003588,-0.014841,-0.009525,0.700565,-0.199375,0.166143,0.748908,...,0.787413,-0.027344,-0.017170,-0.021483,-0.018075,-0.103453,-0.117142,-0.116292,-0.088789,0
7750,-0.404058,-0.504767,-0.013408,-0.010604,-0.033019,-0.009489,-0.114099,1.409368,0.168180,-0.235992,...,-0.994008,-0.121757,-0.080302,-0.133115,-0.106745,-0.431208,-0.130952,-0.438555,-0.415812,0


In [309]:
X = df.drop(['Label'],axis=1) .values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)
pd.Series(y).value_counts()

,count
1,15264
0,10434


### Feature engineering (IG, FCBF, and KPCA)

#### Feature selection by information gain (IG)

In [310]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X, y)

In [311]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [312]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break

In [313]:
X_fs = df[fs].values

In [314]:
X_fs.shape

(25698, 45)

In [315]:
X_fs

array([[ 3.15198492, -0.50912721, -0.39993013, ..., -0.09309893,
        -0.14736467, -0.03859406],
       [ 3.38238041, -0.51970198, -0.40007661, ..., -0.09309893,
        -0.16653457, -0.03855509],
       [ 3.34249182, -0.51744394, -0.40005117, ..., -0.09309893,
        -0.14986471, -0.0385491 ],
       ...,
       [-0.40230915, -0.32083723, -0.385688  , ..., -0.09297328,
         0.74890775, -0.03856708],
       [-0.40405751, -0.46443298, -0.39854351, ..., -0.09309833,
        -0.2359923 , -0.03859346],
       [-0.40230915,  0.75839226,  0.12096069, ..., -0.09309863,
        -0.05356875, -0.03859466]])

#### Feature selection by Fast Correlation Based Filter (FCBF)

The module is imported from the GitHub repo: https://github.com/SantiagoEG/FCBF_module

In [35]:
!pip install scikit-optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.4 MB/s eta 0:00:00


In [316]:
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def fcbf_objective(threshold, X, y, clf=RandomForestClassifier(random_state=42)):
    selector = FCBF(th=threshold)
    X_selected = selector.fit_transform(X, y)
    if X_selected.shape[1] == 0:
        return 1.0
    score = cross_val_score(clf, X_selected, y, cv=3, scoring="accuracy").mean()
    return -score

In [317]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBF(th = 0.09988304703442026)

In [318]:
X_fss = fcbf.fit_transform(X_fs,y)

In [319]:
X_fss.shape

(25698, 1)

In [320]:
X_fss

array([[ 3.15198492],
       [ 3.38238041],
       [ 3.34249182],
       ...,
       [-0.40230915],
       [-0.40405751],
       [-0.40230915]])

####  kernel principal component analysis (KPCA)

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def kpca_objective(params, X, y):
    n_components, kernel = params
    n_components = min(n_components, X.shape[1] - 1)
    if n_components < 1: n_components = 1
    try:
        kpca = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=False, random_state=42)
        X_kpca = kpca.fit_transform(X)
        clf = RandomForestClassifier(random_state=42)
        score = cross_val_score(clf, X_kpca, y, cv=3, scoring="accuracy").mean()
        return -score
    except Exception as e:
        return 1.0

search_space = [
    Integer(10, 50),
    Categorical(['rbf', 'poly'])
]

result = gp_minimize(
    lambda params: kpca_objective(params, X_fss, y),
    search_space,
    n_calls=20,
    random_state=42,
    acq_func='EI'
)

best_n_components, best_kernel = result.x
print(f"Best n_components: {best_n_components}, Best kernel: {best_kernel}")
print(f"Best accuracy: {-result.fun:.4f}")

In [321]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

kpca = KernelPCA(n_components=20, kernel='rbf', random_state=42)
X_kpca = kpca.fit_transform(X_fss)

In [322]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

In [323]:
pd.Series(y_train).value_counts()

,count
1,15056
0,10226


In [324]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15056})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [325]:
pd.Series(y_train).value_counts()

,count
0,15056
1,15056


In [326]:
pd.Series(y_test).value_counts()

,count
1,208
0,208


### Apply the cluster labeling (CL) k-means method

In [286]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN,MeanShift
from sklearn.cluster import SpectralClustering,AgglomerativeClustering,AffinityPropagation,Birch,MiniBatchKMeans,MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [287]:
def CL_kmeans(X_train, X_test, y_train, y_test,n,b=100):
    km_cluster = MiniBatchKMeans(n_clusters=n,batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_test)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm=confusion_matrix(y_test,result2)
    acc=metrics.accuracy_score(y_test,result2)
    print(str(acc))
    print(cm)

In [327]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15056})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [328]:
CL_kmeans(X_train, X_test, y_train, y_test, 8)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       208
           1       0.97      1.00      0.98       208

    accuracy                           0.98       416
   macro avg       0.98      0.98      0.98       416
weighted avg       0.98      0.98      0.98       416

0.9831730769230769
[[201   7]
 [  0 208]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [329]:
#Hyperparameter optimization by BO-GP
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space  = [Integer(2, 50, name='n_clusters')]
@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n=params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_val)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_val)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    cm=metrics.accuracy_score(y_val,result2)
    print(str(n)+" "+str(cm))
    return (1-cm)
from skopt import gp_minimize
import time
t1=time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2=time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.8955902709116077
43 0.9268340913585129
43 0.9385010876013447
43 0.9351394107178169
32 0.8916353569309867
20 0.8947992881154835
16 0.8894601542416453
5 0.497923670160174
15 0.9171445521059917
25 0.9262408542614198
46 0.9386988333003757
50 0.9329642080284754
38 0.9315799881352581
48 0.9345461736207238
40 0.8991496934941665
46 0.9394898160965
36 0.9305912596401028
12 0.5592248368597983
23 0.8847142574649002
27 0.8898556456397073
5.384040117263794
Best score=0.9395
Best parameters: n_clusters=46


In [330]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15056})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [331]:
CL_kmeans(X_train, X_test, y_train, y_test, res_gp.x[0])

              precision    recall  f1-score   support

           0       1.00      0.81      0.89       208
           1       0.84      1.00      0.91       208

    accuracy                           0.90       416
   macro avg       0.92      0.90      0.90       416
weighted avg       0.92      0.90      0.90       416

0.9038461538461539
[[168  40]
 [  0 208]]


### Apply the CL-k-means model with biased classifiers

95% of the code has been shared, and the remaining 5% is retained for future extension.  
Thank you for your interest and more details are in the paper.

In [332]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer
from imblearn.over_sampling import SMOTE

def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    km = MiniBatchKMeans(n_clusters=n, batch_size=b, random_state=0)
    train_labels = km.fit_predict(X_train)
    test_labels  = km.predict(X_test)

    counts_pos = np.zeros(n)
    counts_neg = np.zeros(n)
    for idx, c in enumerate(train_labels):
        if y_train[idx] == 1: counts_pos[c] += 1
        else:               counts_neg[c] += 1

    cluster_prob = {}
    normal_clusters = []
    attack_clusters = []
    for c in range(n):
        tot = counts_pos[c] + counts_neg[c]
        if counts_pos[c] > counts_neg[c]:
            attack_clusters.append(c)
            cluster_prob[c] = counts_pos[c] / tot if tot>0 else 0.0
        else:
            normal_clusters.append(c)
            cluster_prob[c] = counts_neg[c] / tot if tot>0 else 0.0

    y_km = np.array([1 if c in attack_clusters else 0 for c in test_labels])

    print("CL-k-means Performance:")
    print(classification_report(y_test, y_km))
    cm = confusion_matrix(y_test, y_km)
    tn, fp, fn, tp = cm.ravel()
    dr = tp/(tp+fn) if tp+fn>0 else 0
    far = fp/(fp+tn) if fp+tn>0 else 0
    print(f"  DR: {dr:.4f}, FAR: {far:.4f}\n  CM:\n{cm}\n")

    y_train_km = np.array([1 if c in attack_clusters else 0 for c in train_labels])
    fp_idx = np.where((y_train_km == 1) & (y_train == 0))[0]
    fn_idx = np.where((y_train_km == 0) & (y_train == 1))[0]

    X_fp = X_train[fp_idx]
    X_fn = X_train[fn_idx]
    X_norm = X_train[y_train == 0]
    X_adv  = X_train[y_train == 1]

    if len(X_fp) and len(X_adv):
        attack_samples_for_fp = X_adv[np.random.choice(len(X_adv), size=len(X_fp),replace=True)]
        Xp = np.concatenate([X_fp, attack_samples_for_fp])
        yp = np.concatenate([np.zeros(len(X_fp)), np.ones(len(X_fp))])
        yp = yp.astype(np.int64)
        y_counts = np.bincount(yp)
        print(y_counts)
        min_groups = np.min(y_counts)
        if min_groups >= 2:
          opt_rfp = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfp.fit(Xp, yp)
          rfp = opt_rfp.best_estimator_
        else:
          rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    else:
        rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)

    if len(X_fn) and len(X_norm):
        normal_samples_for_fn = X_norm[np.random.choice(len(X_norm), size=len(X_fn),replace=True)]
        Xn = np.concatenate([X_fn, normal_samples_for_fn])
        yn = np.concatenate([np.zeros(len(X_fn)), np.ones(len(X_fn))])
        yn = yn.astype(np.int64)
        y_counts = np.bincount(yn)
        print(y_counts)
        min_groups = np.min(y_counts)

        if min_groups >= 2:
          opt_rfn = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfn.fit(Xn, yn)
          rfn = opt_rfn.best_estimator_
        else:
          rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback
    else:
        rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback

    probs = np.array([cluster_prob.get(c,0.0) for c in test_labels])
    best_thr, best_rec = 0.5, 0.0
    for thr in np.linspace(0.5, 0.99, 50):
        y_tmp = y_km.copy()
        for i, p in enumerate(probs):
            if p < thr:
                if y_tmp[i] == 0:
                    y_tmp[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
                else:
                    y_tmp[i] = rfp.predict(X_test[i].reshape(1,-1))[0]
        rec = recall_score(y_test, y_tmp)
        if rec > best_rec:
            best_rec, best_thr = rec, thr

    y_final = y_km.copy()
    for i, p in enumerate(probs):
        if p < best_thr:
            if y_final[i] == 0:
                y_final[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
            else:
                y_final[i] = rfp.predict(X_test[i].reshape(1,-1))[0]


    print("MTH-IDS Performance:")
    print(classification_report(y_test, y_final))
    cm2 = confusion_matrix(y_test, y_final)

    tn, fp, fn, tp = cm2.ravel()
    dr2 = tp/(tp+fn) if tp+fn>0 else 0
    far2 = fp/(fp+tn) if fp+tn>0 else 0
    acc2 = accuracy_score(y_test, y_final)

    print(f"  Acc: {acc2:.4f}, DR: {dr2:.4f}, FAR: {far2:.4f}\n  CM:\n{cm2}")

    return acc2, dr2, far2, cm2

In [333]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15056})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [334]:
one_acc, one_dr, one_far, one_f1 = Anomaly_IDS(X_train, X_test, y_train, y_test, res_gp.x[0])

CL-k-means Performance:
              precision    recall  f1-score   support

           0       1.00      0.83      0.91       208
           1       0.86      1.00      0.92       208

    accuracy                           0.92       416
   macro avg       0.93      0.92      0.92       416
weighted avg       0.93      0.92      0.92       416

  DR: 1.0000, FAR: 0.1683
  CM:
[[173  35]
 [  0 208]]

[1215 1215]
[761 761]
MTH-IDS Performance:
              precision    recall  f1-score   support

           0       1.00      0.83      0.91       208
           1       0.86      1.00      0.92       208

    accuracy                           0.92       416
   macro avg       0.93      0.92      0.92       416
weighted avg       0.93      0.92      0.92       416

  Acc: 0.9159, DR: 1.0000, FAR: 0.1683
  CM:
[[173  35]
 [  0 208]]
